<a href="https://colab.research.google.com/github/mikedparrott/micro-lessons/blob/master/Connect_to_AWS_Database_Using_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Micro Course:
##**Create an SQL Database & Connect to it with R or Python**

Use the following notebooks to build an SQL database on AWS and connect to it with R or Python. See the [micro lessons github repo](https://github.com/mikedparrott/micro-lessons) for other useful micro lessons.
- [Create an SQL Database on Amazon Web Services](https://github.com/mikedparrott/micro-lessons/blob/master/Create_an_SQL_Database_on_Amazon_Web_Services.ipynb)
- [Connect to AWS Database Using R](/notebooks/mlcc/first_steps_with_tensor_flow.ipynb)
- [Connect to AWS Database Using Python](/notebooks/mlcc/intro_to_neural_nets.ipynb)
- [Delete AWS Database](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)


---

## Micro Lesson II. Connect to AWS Database Using R

###**1.   Sign up for an Amazon Web Services (AWS) account [here](https://aws.amazon.com/) if you do not have an account.**

###**2.   Sign in to your AWS account.  Then click the services tab, search for "RDS", click the "RDS" link to open Amazon's Relational Database Service** 

>(See example GIF below):


![](https://raw.githubusercontent.com/mikedparrott/micro-lessons/master/images/navmysql.png)